<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

In [ ]:
!pip install pyspark

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [37]:
#edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges-test.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [38]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 826
Nodes: 23036


Q1. 

For each drug, compute the number of genes
and the number of diseases associated with the
drug. Output results with top 5 number of genes in a descending orde

In [88]:
# Filter out the Compounds associated with Genes & Diseases
compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
        |
        edges.target.startswith('Disease')
    )
)

In [89]:
compounds.sample(.01).show(10)

+-----------------+--------+------------------+
|           source|metaedge|            target|
+-----------------+--------+------------------+
|Compound::DB00993|     CbG|        Gene::7498|
|Compound::DB00991|     CbG|        Gene::5742|
|Compound::DB00997|     CuG|       Gene::23161|
|Compound::DB00996|     CpD|Disease::DOID:3312|
|Compound::DB00997|     CdG|        Gene::9928|
+-----------------+--------+------------------+



In [82]:
# Convert  |source|target::xyz| -> |source|{target_type: 1}|
rdd2 = compounds.rdd.map(lambda x:
                     (x[0], {x[2].split('::')[0]: 1})
                     )
rdd2.take(5)

[('Compound::DB00997', {'Disease': 1}),
 ('Compound::DB00993', {'Disease': 1}),
 ('Compound::DB00997', {'Disease': 1}),
 ('Compound::DB00993', {'Disease': 1}),
 ('Compound::DB00997', {'Disease': 1})]

Find keys where both have either Disease or Gene, and add up the numbers.

With this data:
```
Compound::DB00991 Gene::5743
Compound::DB00991 Gene::5742
Compound::DB00991 Gene::1559
Compound::DB00991 Disease::DOID:7148
Compound::DB00991 Disease::DOID:839
```
 . . .  we should end up with 
 ```
[
  ('Compound::DB00991', {'Gene': 3}),
  ('Compound::DB00991', {'Disease': 2})
]
 ```

8

In [90]:

rdd3 = rdd2.reduceByKey( lambda x, y:
    {'Disease': x.get('Disease', 0) + y.get('Disease', 0)}
    if ('Disease' in x and 'Disease' in y)
    else
    {'Gene': x.get('Gene', 0) + y.get('Gene', 0)}
)
rdd3.collect()

[('Compound::DB00997', {'Gene': 532}),
 ('Compound::DB00993', {'Gene': 28}),
 ('Compound::DB00995', {'Gene': 63}),
 ('Compound::DB00999', {'Gene': 47}),
 ('Compound::DB00990', {'Gene': 51}),
 ('Compound::DB00996', {'Gene': 18}),
 ('Compound::DB00998', {'Gene': 4}),
 ('Compound::DB00992', {'Gene': 1}),
 ('Compound::DB00991', {'Gene': 3})]

In [75]:
rdd3.reduceByKey( lambda x,y: x[1] + y[2] ).take(10)

[('Compound::DB00997', {'Disease': 17}),
 ('Compound::DB00993', {'Disease': 6}),
 ('Compound::DB00995', {'Disease': 2}),
 ('Compound::DB00999', {'Disease': 2}),
 ('Compound::DB00990', {'Disease': 1}),
 ('Compound::DB00996', {'Disease': 8}),
 ('Compound::DB00998', {'Disease': 1}),
 ('Compound::DB00992', {'Gene': 1}),
 ('Compound::DB00991', {'Disease': 2})]

In [17]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [19]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.